# ▼ 크롤링 시작하기

## 1 클래스 정의

In [4]:
class MovieAPI:
    def __init__(self, index_list, movie_list, dir_list, year_list, client_id, client_secret):
        self.index = []
        self.title_li = []
        self.link_li = []
        self.image_li = []
        self.year_li = []
        self.dir_li = []
        self.actor_li = []
        self.rating_li = []
        self.error_index = []
        self.start = 1
        self.params = ''
        self.url_base = 'https://openapi.naver.com/v1/search/movie.xml?query='
        self.index_list = index_list
        self.movie_list = movie_list
        self.dir_list = dir_list
        self.year_list = year_list
        self.client_id = client_id
        self.client_secret = client_secret

    # ▼ 리스트까지만 반환하는 함수
    def get_movie_list(self):
        import urllib.request
        import re

        movie_list = self.movie_list
        year_list = self.year_list

        for i, movie in enumerate(movie_list):
            movie = movie.replace('(애니)','').replace('(다큐)','').replace('감독판','').replace('확장판','')
            movie = re.sub('[^가-힣A-Za-z0-9\s]', '', movie)   # 한글, 영어, 숫자, 공백 빼고 지우기
            encText = urllib.parse.quote(movie)
            self.params = f'"&display=100&start={self.start}&yearfrom={str(int(year_list[i])-1)}&yearto={str(int(year_list[i]))}'
            url = self.url_base + '"' + encText + self.params
            MovieAPI.get_movie(self, i, movie, url)
            
        return [self.index, self.title_li, self.link_li, self.image_li, self.year_li, self.dir_li, self.actor_li, self.rating_li]


    def get_movie(self, request, i, movie, url):
        import urllib.request
        import xmltodict
        import pandas as pd
        import urllib.request
        import time

        index_list = self.index_list
        movie_list = self.movie_list
        dir_list = self.dir_list
        client_id = self.client_id
        client_secret = self.client_secret
        
        time.sleep(0.5)
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            dict_data = xmltodict.parse(response_body.decode('utf-8'), xml_attribs=False)
            search_li = dict_data['rss']['channel']
            if int(dict_data['rss']['channel']['total']) == 0:          # 검색 결과 없으면
                self.error_index.append([index_list[i], 'noResult'])
                params = f'&display=100&start={self.start}'             # 연도 파라미터 지우고
                encText = urllib.parse.quote(movie)
                url = self.url_base + encText + params                  # url 재설정
                movie = movie.replace('X2','엑스 2').replace('사람','남자').replace('디 오리지널','')     # 못 찾았던 일부 영화 설정
                a,b,c,d,e,f,g,h = MovieAPI.get_movie(self, request, i, movie, url)
                self.index.append(a)
                self.title_li.append(b)
                self.link_li.append(c)
                self.image_li.append(d)
                self.year_li.append(e)
                self.dir_li.append(f)
                self.actor_li.append(g)
                self.rating_li.append(h)
            elif 'item' in search_li.keys():
                search_li = search_li['item']
                if type(search_li) == list:
                    for j in range(len(search_li)):
                        mv_title = search_li[j]['title'].replace('</b>','').replace('<b>','')
                        mv_rating = search_li[j]['userRating']
                        mv_dir = search_li[j]['director']
                        mv_dir = mv_dir.split('|')[:-1] if mv_dir != None else None
                        mv_actor = search_li[j]['actor']
                        mv_actor = mv_actor.split('|')[:-1] if mv_actor != None else None
                        mv_year = search_li[j]['pubDate']
                        if mv_title.replace(' ','') == movie.replace(' ','') and mv_rating != None and mv_dir != None and mv_actor != None:
                            if mv_rating != '0.00' and dir_list[i] in mv_dir:
                                self.index.append(index_list[i])
                                self.title_li.append(movie)
                                self.link_li.append(search_li[j]['link'])
                                self.image_li.append(search_li[j]['image'])
                                self.year_li.append(mv_year)
                                self.dir_li.append(mv_dir)
                                self.actor_li.append(mv_actor)
                                self.rating_li.append(mv_rating)
                                break
                            elif self.index != [] and self.index[-1] != i and mv_rating != '0.00' and dir_list[i] not in mv_dir:
                                if int(dict_data['rss']['channel']['total']) <= 100:
                                    self.index.append(index_list[i])
                                    self.title_li.append(movie)
                                    self.link_li.append(search_li[j]['link'])
                                    self.image_li.append(search_li[j]['image'])
                                    self.year_li.append(search_li[j]['pubDate'])
                                    self.dir_li.append(mv_dir)
                                    self.actor_li.append(mv_actor)
                                    self.rating_li.append(mv_rating)
                                else:
                                    self.error_index.append([index_list[i], '3rd'])
                                    self.start += 1
                                    a,b,c,d,e,f,g,h = MovieAPI.get_movie(self, request, i, movie, url)
                                    self.index.append(a)
                                    self.title_li.append(b)
                                    self.link_li.append(c)
                                    self.image_li.append(d)
                                    self.year_li.append(e)
                                    self.dir_li.append(f)
                                    self.actor_li.append(g)
                                    self.rating_li.append(h)
                                    self.start = 1
                            else:
                                self.error_index.append([index_list[i], '2nd'])
                                break
                        else:
                            self.error_index.append([index_list[i], '1st'])
                            continue
                elif type(search_li) == dict:
                    mv_title = search_li['title'].replace('</b>','').replace('<b>','')
                    mv_dir = search_li['director']
                    mv_dir = mv_dir.split('|')[:-1] if mv_dir != None else None
                    mv_actor = search_li['actor']
                    mv_actor = mv_actor.split('|')[:-1] if mv_actor != None else None

                    self.index.append(index_list[i])
                    self.title_li.append(movie)
                    self.link_li.append(search_li['link'])
                    self.image_li.append(search_li['image'])
                    self.year_li.append(search_li['pubDate'])
                    self.dir_li.append(mv_dir)
                    self.actor_li.append(mv_actor)
                    self.rating_li.append(search_li['userRating'])
                else:
                    self.error_index.append([index_list[i], 'list/dict 분류 오류'])
        else:
            print("Error Code:" + rescode)
        if (i != 0 and i % 10 == 0) or i == (len(movie_list)-1):
            print(f'{index_list[i]}번째 완료')
        if i != 0 and i % 500 == 0:    # 500번마다 savepoint로 저장해두기 
            pd.DataFrame([self.index, self.title_li, self.link_li, self.image_li, self.year_li, self.dir_li, self.actor_li, self.rating_li]).transpose().to_csv(f'./files/savepoint{index_list[i]}_df_utf-8.csv', encoding='utf-8')
            print('savepoint', index_list[i])

        return [self.index, self.title_li, self.link_li, self.image_li, self.year_li, self.dir_li, self.actor_li, self.rating_li]

    # ▼ get_movie 실행 및 저장 함수
    def mk_movie_df(self):
        import pandas as pd
        
        li_to_df = MovieAPI.get_movie_list(self)
        movie_df = pd.DataFrame(li_to_df).transpose()
        movie_df.columns = ['순번', '영화명', '네이버URL', '이미지URL', '연도', '감독', '출연', '평점']

        # 결과 데이터 중복 제거
        df_drop_dp = movie_df.drop_duplicates(['순번','영화명'], keep='last')
        df_drop_dp

        # 인덱스 설정
        movie_df = movie_df.set_index('순번')
        df_drop_dp = df_drop_dp.set_index('순번')

        # 결과 데이터와 중복 제거 데이터 저장
        movie_df.to_csv('./files/naver_api_utf-8.csv', encoding='utf-8')
        df_drop_dp.to_csv('./files/naver_api_drop_dupl_utf-8.csv', encoding='utf-8')

        return movie_df, df_drop_dp


## 2 리스트 불러오기

### 2-1 연도 컬럼 만들기(연도 컬럼 없는 경우) 

In [25]:
import pandas as pd

movie_df = pd.read_csv('movie_resize2.csv', encoding='utf-8')
year_df = pd.DataFrame()
for row in movie_df['개봉일']:
    year_df = pd.concat([year_df, pd.DataFrame([row.split('-')[0]])])

year_df.columns = ['연도']

year_df.index = range(4642)

movie_df = pd.read_csv('movie_resize2.csv', encoding='utf-8')
movie_df = pd.concat([movie_df, year_df], axis=1)

movie_df.to_csv('movie_resize2_year_utf8.csv', encoding='utf-8')

,연도
0,2014
0,2019
0,2017
0,2014
0,2019
...,...
0,2015
0,2011
0,2010
0,2019


### 2-2 파일 불러오기
DataFrame 불러와서 '인덱스', '영화명', '감독', '연도' 리스트로 각자 넣어주기

In [10]:
import pandas as pd

# 리스트 불러오기
movie_df = pd.read_csv('movie_resize2_year_utf8.csv', encoding='utf-8', index_col=0)
display(movie_df[:6])
index_list = list(movie_df.index)
movie_list = movie_df['영화명'].to_list()
dir_list = movie_df['감독'].to_list()
year_list = movie_df['연도'].to_list()

print('len =',len(index_list,), index_list[:10])
print('len =',len(movie_list), movie_list[:10])
print('len =',len(dir_list), dir_list[:10])
print('len =',len(year_list), year_list[:10])


,영화명,감독,배급사,개봉일,영화형태,국적,전국스크린수,전국 매출액,전국 관객수,장르,등급,영화구분,연도
0,명량,김한민,(주)씨제이이엔엠,2014-07-30,장편,한국,1587,135748398910,17613682,사극,15세관람가,일반영화,2014
1,극한직업,이병헌,(주)씨제이이엔엠,2019-01-23,장편,한국,1978,139647979516,16264944,코미디,15세관람가,일반영화,2019
2,신과함께-죄와 벌,김용화,롯데쇼핑㈜롯데엔터테인먼트,2017-12-20,장편,한국,1912,115698654137,14410754,판타지,12세관람가,일반영화,2017
3,국제시장,윤제균,(주)씨제이이엔엠,2014-12-17,장편,한국,966,110828014630,14245998,드라마,12세관람가,일반영화,2014
4,어벤져스: 엔드게임,안소니 루소,월트디즈니컴퍼니코리아 유한책임회사,2019-04-24,장편,미국,2835,122182694160,13934592,액션,12세관람가,일반영화,2019
5,겨울왕국 2,제니퍼 리,월트디즈니컴퍼니코리아 유한책임회사,2019-11-21,장편,미국,2648,114810421450,13747792,애니메이션,전체관람가,일반영화,2019


len = 4642 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
len = 4642 ['명량', '극한직업', '신과함께-죄와 벌', '국제시장', '어벤져스: 엔드게임', '겨울왕국 2', '아바타', '베테랑', '괴물', '도둑들']
len = 4642 ['김한민', '이병헌', '김용화', '윤제균', '안소니 루소', '제니퍼 리', '제임스 카메론', '류승완', '봉준호', '최동훈']
len = 4642 [2014, 2019, 2017, 2014, 2019, 2019, 2009, 2015, 2006, 2012]


## 3 함수 실행하기

### 3-1 변수 할당 및 실행

In [14]:
# id, secret 입력
client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

# 함수 실행
# MovieAPI 클래스를 ma 변수에 할당
ma = MovieAPI(index_list=index_list, movie_list=movie_list, dir_list=dir_list, year_list=year_list, client_id=client_id, client_secret=client_secret)
# 수집된 전체 df와 중복 제거한 df를 각각의 변수에 할당, mk_movie_df 함수 실행
nv_df, nv_df_drop_dp = ma.mk_movie_df()

display(nv_df.head())

10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
60번째 완료
70번째 완료
80번째 완료
90번째 완료
100번째 완료
110번째 완료
120번째 완료
130번째 완료
140번째 완료
150번째 완료
160번째 완료
170번째 완료
180번째 완료
190번째 완료
200번째 완료
210번째 완료
220번째 완료
230번째 완료
240번째 완료
250번째 완료
260번째 완료
270번째 완료
280번째 완료
290번째 완료
300번째 완료
310번째 완료
320번째 완료
330번째 완료
340번째 완료
350번째 완료
360번째 완료
370번째 완료
380번째 완료
390번째 완료
400번째 완료
410번째 완료
420번째 완료
430번째 완료
440번째 완료
450번째 완료
460번째 완료
470번째 완료
480번째 완료
490번째 완료
500번째 완료
savepoint 500
510번째 완료
520번째 완료
530번째 완료
540번째 완료
550번째 완료
560번째 완료
570번째 완료
580번째 완료
590번째 완료
600번째 완료
610번째 완료
620번째 완료
630번째 완료
640번째 완료
650번째 완료
660번째 완료
670번째 완료
680번째 완료
690번째 완료
700번째 완료
710번째 완료
720번째 완료
730번째 완료
740번째 완료
750번째 완료
760번째 완료
770번째 완료
780번째 완료
790번째 완료
800번째 완료
810번째 완료
820번째 완료
830번째 완료
840번째 완료
850번째 완료
860번째 완료
870번째 완료
880번째 완료
890번째 완료
900번째 완료
910번째 완료
920번째 완료
930번째 완료
940번째 완료
950번째 완료
960번째 완료
970번째 완료
980번째 완료
990번째 완료
1000번째 완료
savepoint 1000
1010번째 완료
1020번째 완료
1030번째 완료
1040번째 완료
1050번째 완료
1060번째 완료
1070번째 완료
1080번째 완료

c:\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


### 3-2 누락 갯수 확인하기

In [16]:
len(movie_df) - len(nv_df)

370

### 3-3 누락 리스트 확인하기

In [16]:
import numpy as np

aa = nv_df.index.to_numpy()             # 추출 인덱스 (ndarray 1차원)
bb = np.array(range(0,4642))            # 원본 인덱스 (ndarray 1차원)

diff_index = np.setdiff1d(bb, aa)       # 원본 인덱스(bb)에서 추출 인덱스(aa) 빼기
missed_df = movie_df.loc[diff_index]    # 원본 파일(movie_df)에서 해당 인덱스의 DataFrame 뽑아내기

missed_df.to_csv('./files/missed_utf8.csv', encoding='utf-8')
missed_df.head()

,영화명,감독,배급사,개봉일,영화형태,국적,전국스크린수,전국 매출액,전국 관객수,장르,등급,영화구분,연도
644,해적: 도깨비 깃발,김정훈,롯데컬처웍스(주)롯데엔터테인먼트,2022-01-26,장편,한국,1708,12523963990,1339242,어드벤처,12세관람가,일반영화,2022
652,월ㆍE,앤드류 스탠튼,한국소니픽쳐스릴리징브에나비스타영화㈜,2008-08-06,장편,미국,347,8264916667,1320830,애니메이션,전체관람가,일반영화,2008
721,인생은 아름다워,최국희,롯데컬처웍스(주)롯데엔터테인먼트,2022-09-28,장편,한국,1285,10898590019,1171648,뮤지컬,12세관람가,일반영화,2022
894,블랙,산제이 렐라 반살리,이언픽쳐스,2009-08-27,장편,인도,248,6254367609,874913,드라마,전체관람가,독립/예술영화,2009
930,데자뷰,토니 스콧,한국소니픽쳐스릴리징브에나비스타영화㈜,2007-01-11,장편,미국,0,0,829891,액션,12세관람가,일반영화,2007


In [ ]:
1191 : 트리플X2넥스트레벨 (2005)
 : 애프터선셋 (2005)
 : 스페이스침스우주선을찾아서 (2008)
 : 헨젤과그레텔마녀사냥꾼3D (2013)
 : pm1114 (2005)
 : 패스트앤퓨리어스도쿄드리프트 (2006)
 : 링22005 (2005)
 : 독전익스텐디드컷 (2018)
 : 아더와미니모이제1탄비밀원정대의출정 (2009)
 : 킹오브프리즘바이프리티리듬 (2016)
 : 펭귄위대한모험다큐 (2005)
 : 얼음왕국북극의여름이야기다큐 (2006)
 : 40살까지못해본사람 (2005)
 : 마하26풀스피드 (2007)
 : 반지의제왕반지원정대확장판 (2017)
 : 반지의제왕왕의귀환확장판 (2017)
 : 반지의제왕두개의탑확장판 (2017)
 : 4요일자살을부르는요일 (2008)
 : 러빙빈센트비하인드에디션 (2018)
 : 비정한도시감독판 (2012)
 : 그랑블루리마스터링감독판 (2013)
 : 맨츄리안캔디데이트 (2005)
 : 강철비2정상회담확장판 (2020)
 : 2008서태지심포니위드톨가카쉬프로열필하모닉 (2010)
 : 아마데우스감독판 (2015)
 : 뜨거운피디오리지널 (2022)
 : 꼬마비행기피티와친구들사막구출대작전 (2018)
 : 프리더파이널스트로크전편 (2022)
 : 써드퍼슨감독판 (2015)
 : 극장판빰바라빤쮸바나나왕국의비밀 (2017)
[1191,1539,1688,1689,1728,1842,1886,2174,2233,2403,2554,2728,2859,2967,3275,3413,3504,3521,3571,3695,3763,3821,3911,3959,4163,4217,4229,4292,4518,4545]

# ▼ 아래는 함수 오류 확인용

### 0 준비하기

In [5]:
import pandas as pd

# 리스트 불러오기
missed_df = pd.read_csv('./files/missed_utf8.csv', encoding='utf-8', index_col=0)
display(missed_df[:6])
index_list_m = list(missed_df.index)
movie_list_m = missed_df['영화명'].to_list()
dir_list_m = missed_df['감독'].to_list()
year_list_m = missed_df['연도'].to_list()

print('len =',len(index_list_m), index_list[:10])
print('len =',len(movie_list_m), movie_list[:10])
print('len =',len(dir_list_m), dir_list[:10])
print('len =',len(year_list_m), year_list[:10])

,영화명,감독,배급사,개봉일,영화형태,국적,전국스크린수,전국 매출액,전국 관객수,장르,등급,영화구분,연도
644,해적: 도깨비 깃발,김정훈,롯데컬처웍스(주)롯데엔터테인먼트,2022-01-26,장편,한국,1708,12523963990,1339242,어드벤처,12세관람가,일반영화,2022
652,월ㆍE,앤드류 스탠튼,한국소니픽쳐스릴리징브에나비스타영화㈜,2008-08-06,장편,미국,347,8264916667,1320830,애니메이션,전체관람가,일반영화,2008
721,인생은 아름다워,최국희,롯데컬처웍스(주)롯데엔터테인먼트,2022-09-28,장편,한국,1285,10898590019,1171648,뮤지컬,12세관람가,일반영화,2022
894,블랙,산제이 렐라 반살리,이언픽쳐스,2009-08-27,장편,인도,248,6254367609,874913,드라마,전체관람가,독립/예술영화,2009
930,데자뷰,토니 스콧,한국소니픽쳐스릴리징브에나비스타영화㈜,2007-01-11,장편,미국,0,0,829891,액션,12세관람가,일반영화,2007
932,로봇(애니),카를로스 살다나,이십세기폭스필름코퍼레이션,2005-07-28,장편,미국,0,0,826100,코미디,전체관람가,일반영화,2005


len = 375 [644, 652, 721, 894, 930, 932, 1093, 1145, 1191, 1203]
len = 375 ['해적: 도깨비 깃발', '월ㆍE', '인생은 아름다워', '블랙', '데자뷰', '로봇(애니)', '지푸라기라도 잡고 싶은 짐승들', '극장판 포켓몬스터DP: 기라티나와 하늘의 꽃다발 쉐이미', '트리플 X2 : 넥스트 레벨', '이상한 나라의 수학자']
len = 375 ['김정훈', '앤드류 스탠튼', '최국희', '산제이 렐라 반살리', '토니 스콧', '카를로스 살다나', '김용훈', '유야마 쿠니히코', '리 타마호리', '박동훈']
len = 375 [2022, 2008, 2022, 2009, 2007, 2005, 2020, 2022, 2005, 2022]


### 1 xml 형태 확인

In [19]:
client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

import urllib.request
import xmltodict
import pandas as pd
import time
import re

index = index_list_m
year_list = year_list_m
dir_list = dir_list_m
for i, movie in enumerate(movie_list_m):
    movie = movie.replace('(애니)','').replace('(다큐)','').replace('감독판','').replace('확장판','').replace('디 오리지널','')
    movie = re.sub('[^가-힣A-Za-z0-9]|\s', ' ', movie)
    encText = urllib.parse.quote(movie)
    url = 'https://openapi.naver.com/v1/search/movie.xml?query=' + encText  + f'&display=100'
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        dict_data = xmltodict.parse(response_body.decode('utf-8'), xml_attribs=False)
        if int(dict_data['rss']['channel']['total']) == 0:
            print(index[i], ':', movie, f'({year_list[i]})')
            movie = movie.replace('X2','엑스 2').replace('사람','남자')

1191 : 트리플 X2   넥스트 레벨 (2005)
2554 : 펭귄   위대한 모험 다큐  (2005)
2728 : 얼음왕국   북극의 여름이야기 다큐  (2006)
2859 : 40살까지 못해본 사람 (2005)
3521 : 4요일  자살을 부르는 요일 (2008)
3959 : 2008 서태지 심포니 위드 톨가 카쉬프 로열필하모닉 (2010)
4217 : 뜨거운 피  디 오리지널 (2022)


In [259]:
print(url)

https://openapi.naver.com/v1/search/movie.xml?query="%EB%A7%88%EB%8D%94"&display=100&yearfrom=2007&yearto=2011


### 2 함수 실행해서 확인

In [12]:
client_id = "viO54DlhW87ESywUDwWb"
client_secret = "7ZHAscDgKs"

ma = MovieAPI()
naver_df = ma.get_movie(movie_list=['월드 워 Z', '마더', '친구 2'], dir_list=['샘 햄슨', '봉준호', '곽경택'], year_list=['2013', '2009', '2013'], client_id=client_id, client_secret=client_secret)

# naver_df.to_csv('naver_api_utf-8_missed2.csv', encoding='utf-8')
display(naver_df)

year_list=['2013', '2009', '2013']
df_dropped = naver_df.drop_duplicates(['영화명'], keep='last')
df_dropped

2번째 완료


c:\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


,영화명,네이버URL,이미지URL,연도,감독,출연,평점
순번,,,,,,,
0,월드워Z,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/05...,2013,[마크 포스터],"[브래드 피트, 미레유 에노스]",8.24
1,마더,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/04...,2009,[봉준호],"[김혜자, 원빈]",8.19
2,친구2,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/10...,2013,[곽경택],"[유오성, 주진모, 김우빈]",6.20


,영화명,네이버URL,이미지URL,연도,감독,출연,평점
순번,,,,,,,
0,월드워Z,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/05...,2013,[마크 포스터],"[브래드 피트, 미레유 에노스]",8.24
1,마더,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/04...,2009,[봉준호],"[김혜자, 원빈]",8.19
2,친구2,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/10...,2013,[곽경택],"[유오성, 주진모, 김우빈]",6.20


In [ ]:
# https://jimmy-ai.tistory.com/102
# df.drop_duplicates()

# ⊙ 맨 아래